Emotive for LSL realtime

https://www.emotiv.com/blogs/tutorials/emotiv-lab-streaming-layer-lsl

FFT

https://github.com/KrishnaVeer7712/DIGITS-CLASSIFICATIONS-USING-ELECTROENCEPHALOGRAPHY-SIGNALS/blob/master/Brain_123%20(1).ipynb

In [12]:
import mne
import os
import numpy as np

root_folder = "./data/emotiv/Digit"

X = np.zeros((230,14,1280))
Y = np.zeros((230,))

ctr = 0

for file in os.listdir(root_folder):
    if file.endswith(".edf"):
        data = mne.io.read_raw_edf(os.path.join(root_folder,file))
        raw_data = data[2:16][0]*1000
        raw_data = raw_data[:,0:1280]
        #a = raw_data.shape
        
        _,cls = file.split('_')
        Y[ctr] = int(cls[0])
        X[ctr,:,:] = raw_data
        ctr = ctr+1

Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data/emotiv/Digit/shubhamtiwari_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data/emotiv/Digit/Vipin_7.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data/emotiv/Digit/Rajkumarsaini_7.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data/emotiv/Digit/ChandanSingha_4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data/emotiv/Digit/Nishant_9.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/hunkim/Github/studio-brain/data

In [13]:
X.shape,Y.shape

((230, 14, 1280), (230,))

In [ ]:
X[0]


array([[4.24974359, 4.25179487, 4.25333333, ..., 4.23076923, 4.23589744,
        4.23846154],
       [4.25435897, 4.25692308, 4.25589744, ..., 4.22666667, 4.22871795,
        4.23589744],
       [4.25794872, 4.26461538, 4.26410256, ..., 4.21538462, 4.22666667,
        4.23128205],
       ...,
       [4.22974359, 4.23179487, 4.22871795, ..., 4.22461538, 4.22358974,
        4.22461538],
       [4.2625641 , 4.27487179, 4.27641026, ..., 4.22769231, 4.22769231,
        4.22410256],
       [4.26923077, 4.2774359 , 4.27897436, ..., 4.20871795, 4.21538462,
        4.22512821]])

In [20]:
# find unique value of Y
np.unique(Y)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
X_new = np.zeros((36110,32,14))
Y_new = np.zeros((36110,))
npt = 32
stride = 8
ctr = 0

row_count = X.shape[0]
for i in range(0,row_count):
    # We examine each row of the data
    y = Y[i]
    a= X[i,:,:]
    a = a.transpose()
    val = 0
    while val<=(len(a)-npt):
        x = a[val:val+npt,:]
        X_new[ctr,:,:] = x
        Y_new[ctr] = y
        val = val+stride
        ctr = ctr+1